In [ ]:
from sklearn.neighbors import NearestNeighbors, kneighbors_graph
from sklearn.manifold import trustworthiness
from scipy.sparse.csgraph import dijkstra
from scipy.spatial import distance_matrix
from sklearn.manifold import MDS
import matplotlib.pyplot as plt
import multiprocessing as mp
from tqdm.auto import tqdm
import tensorflow as tf
import numpy as np
import random
import queue
import json
		

# Antenna definitions
ASSIGNMENTS = [
	[0, 13, 31, 29, 3, 7, 1, 12 ],
	[30, 26, 21, 25, 24, 8, 22, 15],
	[28, 5, 10, 14, 6, 2, 16, 18],
	[19, 4, 23, 17, 20, 11, 9, 27]
]

ANTENNACOUNT = np.sum([len(antennaArray) for antennaArray in ASSIGNMENTS])

def load_calibrate_timedomain(path, offset_path):
	offsets = None
	with open(offset_path, "r") as offsetfile:
		offsets = json.load(offsetfile)
	
	def record_parse_function(proto):
		record = tf.io.parse_single_example(
			proto,
			{
				"csi": tf.io.FixedLenFeature([], tf.string, default_value=""),
				"pos-tachy": tf.io.FixedLenFeature([], tf.string, default_value=""),
				"time": tf.io.FixedLenFeature([], tf.float32, default_value=0),
			},
		)

		csi = tf.ensure_shape(tf.io.parse_tensor(record["csi"], out_type=tf.float32), (ANTENNACOUNT, 1024, 2))
		csi = tf.complex(csi[:, :, 0], csi[:, :, 1])
		csi = tf.signal.fftshift(csi, axes=1)

		position = tf.ensure_shape(tf.io.parse_tensor(record["pos-tachy"], out_type=tf.float64), (3))
		time = tf.ensure_shape(record["time"], ())

		return csi, position[:2], time

	def apply_calibration(csi, pos, time):
		sto_offset = tf.tensordot(tf.constant(offsets["sto"]), 2 * np.pi * tf.range(tf.shape(csi)[1], dtype = np.float32) / tf.cast(tf.shape(csi)[1], np.float32), axes = 0)
		cpo_offset = tf.tensordot(tf.constant(offsets["cpo"]), tf.ones(tf.shape(csi)[1], dtype = np.float32), axes = 0)
		csi = tf.multiply(csi, tf.exp(tf.complex(0.0, sto_offset + cpo_offset)))

		return csi, pos, time

	def csi_time_domain(csi, pos, time):
		csi = tf.signal.fftshift(tf.signal.ifft(tf.signal.fftshift(csi, axes=1)),axes=1)

		return csi, pos, time

	def cut_out_taps(tap_start, tap_stop):
		def cut_out_taps_func(csi, pos, time):
			return csi[:,tap_start:tap_stop], pos, time

		return cut_out_taps_func

	def order_by_antenna_assignments(csi, pos, time):
		csi = tf.stack([tf.gather(csi, antenna_inidces) for antenna_inidces in ASSIGNMENTS])
		return csi, pos, time
	
	dataset = tf.data.TFRecordDataset(path)
	
	dataset = dataset.map(record_parse_function, num_parallel_calls = tf.data.AUTOTUNE)
	dataset = dataset.map(apply_calibration, num_parallel_calls = tf.data.AUTOTUNE)
	dataset = dataset.map(csi_time_domain, num_parallel_calls = tf.data.AUTOTUNE)
	dataset = dataset.map(cut_out_taps(507, 520), num_parallel_calls = tf.data.AUTOTUNE)
	dataset = dataset.map(order_by_antenna_assignments, num_parallel_calls = tf.data.AUTOTUNE)

	return dataset

inputpaths = [
	{
		"tfrecords" : "dichasus/dichasus-cf02.tfrecords",
		"offsets" : "dichasus/reftx-offsets-dichasus-cf02.json"
	},
	{
		"tfrecords" : "dichasus/dichasus-cf03.tfrecords",
		"offsets" : "dichasus/reftx-offsets-dichasus-cf03.json"
	},
	{
		"tfrecords" : "dichasus/dichasus-cf04.tfrecords",
		"offsets" : "dichasus/reftx-offsets-dichasus-cf04.json"
	}
]

full_dataset = load_calibrate_timedomain(inputpaths[0]["tfrecords"], inputpaths[0]["offsets"])

for path in inputpaths[1:]:
	full_dataset = full_dataset.concatenate(load_calibrate_timedomain(path["tfrecords"], path["offsets"]))
    

In [ ]:
training_set = full_dataset.enumerate().filter(lambda idx, value : (idx % 4 == 0))
training_set = training_set.map(lambda idx, value : value)

In [ ]:
groundtruth_positions = []
csi_time_domain = []
timestamps = []
for csi, pos, time in training_set.batch(1000):
	csi_time_domain.append(csi.numpy())
	groundtruth_positions.append(pos.numpy())
	timestamps.append(time.numpy())

csi_time_domain = np.concatenate(csi_time_domain)
groundtruth_positions = np.concatenate(groundtruth_positions)
timestamps = np.concatenate(timestamps)

In [ ]:
import zipfile
import os

zip_path = "siamese_final_model.zip"  # Update with your actual path
extract_path = "siamese_model_extracted"

# Extract ZIP
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Extraction complete. Files are inside:", extract_path)


In [ ]:
import tensorflow as tf
import os
import numpy as np

# Correct path to your SavedModel directory
model_path = os.path.join("siamese_model_extracted", "siamese_final_model")

# Custom Keras Layer to Load a SavedModel
class CustomTFSMLayer(tf.keras.layers.Layer):
    def __init__(self, model_path, **kwargs):
        super().__init__(**kwargs)
        self.model = tf.saved_model.load(model_path)  # Load SavedModel

    def call(self, inputs):
        # Ensure inputs is a tuple with two tensors: input_1 and input_2
        input_1, input_2 = inputs
        
        # Pass inputs as a dictionary matching the model signature
        inference_fn = self.model.signatures["serving_default"]
        
        # Pass the actual tensors (input_1 and input_2) to the inference function
        return inference_fn(input_1=input_1, input_2=input_2)["concatenate"]

# Instantiate the model layer
model_layer = CustomTFSMLayer(model_path)

print("Model Loaded Successfully!")

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout
from tensorflow.keras.models import Model

def build_denoising_autoencoder(input_dim):
    """
    Builds a denoising autoencoder model.
    
    Args:
        input_dim: Dimension of the input features (e.g., 832 for features_F1).
    
    Returns:
        autoencoder: The denoising autoencoder model.
    """
    # Input layer
    input_data = Input(shape=(input_dim,), name="input")
    
    # Encoder
    x = Dense(512, activation="relu", name="encoder_dense_1")(input_data)
    x = BatchNormalization(name="encoder_bn_1")(x)
    x = Dropout(0.2, name="encoder_dropout_1")(x)
    
    x = Dense(256, activation="relu", name="encoder_dense_2")(x)
    x = BatchNormalization(name="encoder_bn_2")(x)
    x = Dropout(0.2, name="encoder_dropout_2")(x)
    
    x = Dense(128, activation="relu", name="encoder_dense_3")(x)
    x = BatchNormalization(name="encoder_bn_3")(x)
    x = Dropout(0.2, name="encoder_dropout_3")(x)
    
    encoded = Dense(64, activation="relu", name="encoder_latent")(x)  # Latent space
    
    # Decoder
    x = Dense(128, activation="relu", name="decoder_dense_1")(encoded)
    x = BatchNormalization(name="decoder_bn_1")(x)
    x = Dropout(0.2, name="decoder_dropout_1")(x)
    
    x = Dense(256, activation="relu", name="decoder_dense_2")(x)
    x = BatchNormalization(name="decoder_bn_2")(x)
    x = Dropout(0.2, name="decoder_dropout_2")(x)
    
    x = Dense(512, activation="relu", name="decoder_dense_3")(x)
    x = BatchNormalization(name="decoder_bn_3")(x)
    x = Dropout(0.2, name="decoder_dropout_3")(x)
    
    decoded = Dense(input_dim, activation="linear", name="decoder_output")(x)  # Reconstruct clean features
    
    # Autoencoder model
    autoencoder = Model(input_data, decoded, name="DenoisingAutoencoder")
    
    return autoencoder

# Build the denoising autoencoder
input_dim = 416  # Dimension of the input features
autoencoder = build_denoising_autoencoder(input_dim)

# Compile the autoencoder
autoencoder.compile(optimizer="adam", loss="mse")

# Print the autoencoder summary
autoencoder.summary()

In [ ]:
mae_noise_list = []
mae_denoise_list = []
lp_values = []

for x in range(5, 105, 10):
    lp_values.append(x)

    # Parameters
    W = 1024
    modulation_order = 16
    C = W // 8
    L = 13
    Lp = x
    noise_power = 0.1
    B = 32

    # Generate frequency-domain symbols
    data = np.random.randint(0, modulation_order, W)
    qam_symbols = (2 * (data % 4) - 3) + 1j * (2 * (data // 4) - 3)
    qam_symbols /= np.sqrt(10)

    time_domain_signal = np.fft.ifft(qam_symbols)

    # Zero-mean Gaussian for amplitude
    A_k = np.random.normal(loc=0.0, scale=1.0, size=Lp)
    # Zero-mean Gaussian for phase
    phi_k = np.random.normal(loc=0.0, scale=1.0, size=Lp)
    p_bar = A_k * np.exp(1j * phi_k)
    p_bar /= np.linalg.norm(p_bar)

    perturbed_signal = np.convolve(p_bar, time_domain_signal, mode="full")[:W]
    cyclic_prefix = perturbed_signal[-C:]
    transmit_signal = np.concatenate([cyclic_prefix, perturbed_signal])

    csi_time_domain = csi_time_domain.reshape(20997, 32, 13)
    num_csi_instances = csi_time_domain.shape[0]

    y_real_all = np.zeros((num_csi_instances, B, W + C), dtype=complex)
    H_pert_all = np.zeros((num_csi_instances, B, W), dtype=complex)

    for i in range(num_csi_instances):
        H_real = csi_time_domain[i]
        for b in range(B):
            y_real = convolve(H_real[b], transmit_signal, mode='full')[:W + C]
            noise = np.sqrt(noise_power / 2) * (np.random.randn(W + C) + 1j * np.random.randn(W + C))
            y_real_noisy = y_real + noise
            y_real_no_cp = y_real_noisy[C:]

            y_freq = np.fft.fft(y_real_no_cp)
            s_freq = np.fft.fft(time_domain_signal)

            H_pert_all[i, b, :] = y_freq / s_freq
            H_pert_time = np.fft.ifft(H_pert_all[i, b, :])[:L]
            csi_time_domain[i, b, :] = H_pert_time

    csi_time_domain = csi_time_domain.reshape(20997, 4, 8, 13)
    reshaped_csi = csi_time_domain.reshape(20997, 416)
    X_train = reshaped_csi 
    y_train = reshaped_csi 

    history = autoencoder.fit(
        X_train, y_train,
        epochs=30,
        batch_size=32,
        validation_split=0.2,
        verbose=0
    )
    csi_tensor = tf.Variable(tf.convert_to_tensor(csi_time_domain, dtype=tf.complex64))
    x_noise = model_layer([csi_tensor,csi_tensor])
    csi_denoise = autoencoder.predict(reshaped_csi)
    csi_denoise = csi_denoise.reshape(20997, 4, 8, 13)
    csi_tensor_denoise = tf.Variable(tf.convert_to_tensor(csi_denoise, dtype=tf.complex64))
    x_denoise = model_layer([csi_tensor_denoise, csi_tensor_denoise])
    x_noise_transformed = affine_transform_channel_chart(groundtruth_positions, x_noise)
    x_denoise_transformed = affine_transform_channel_chart(groundtruth_positions, x_denoise)
    mae_noise = np.mean(np.abs(x_noise_transformed - groundtruth_positions))
    mae_denosie = np.mean(np.abs(x_denoise_transformed - groundtruth_positions))

    mae_noise_list.append(mae_noise)
    mae_denoise_list.append(mae_denosie)

# Plot MAE vs Perturbation Length
plt.plot(lp_values, mae_noise_list, label='MAE (Noisy Features, Siamese)', marker='o')
plt.plot(lp_values, mae_denoise_list, label='MAE (Denoised Features)', marker='s')
plt.xlabel('Perturbation Length (Lp)')
plt.ylabel('Mean Absolute Error (MAE)')
plt.title('MAE vs Perturbation Length')
plt.legend()
plt.grid(True)
plt.show()